# TODO
- scale 512x512 les images avant de les rentrer dans le boxdataset

In [ ]:
from libs import preprocessing as pp
from libs.preprocessing.dataset import imageDataset
from libs.utils import imageToTensor, crop_imgs, draw_img_boxes
from carDetector import carDetector

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn import preprocessing

In [ ]:
home = "/home/jovyan/activities_data/hi__paris_2022_hackathon/final_challenge/datasets"
car_path = f"{home}/car_models_footprint.csv"
annotation_path = f"{home}/datasets_train/train_annotation/_annotation.csv"
images_path = f"{home}/datasets_train/train"
car_images_path = f"{home}/datasets_train/car_models_database_train/"

In [ ]:
x, y = pp.create_car_dataset(car_path, annotation_path, label="brand")
carDataset = imageDataset(x, images_path)
labelEncoder = preprocessing.LabelEncoder().fit(y["models"])

In [ ]:
X_classifier, y_classifier = pp.create_image_dataset(car_images_path)
y_classifier_id = labelEncoder.transform(y_classifier["label"])
classifierDataset = imageDataset(X_classifier, car_images_path)

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn.functional import one_hot

In [ ]:
def imageCollater(x, size=256, augment=True):
    """
        Resize and collate image together
        
        x: list of images numpy array
    """
    
    # Resize image
    resizer = transforms.Resize(size=size)
    
    # Random crop
    cropper = transforms.CenterCrop((size, size))
    
    # Random flip
    flipper_1 = transforms.RandomHorizontalFlip()
    flipper_2 = transforms.RandomVerticalFlip()
    
    # Random rotate
    rotater = transforms.RandomRotation((-90, 90))
    
    # Application of the operations
    tensors_list = []
    for image in x:
        image = imageToTensor(image)
        
        # Fixing missing axis :
        if image.dim() == 2:
            image = image.repeat(3, 1, 1)
        
        image = resizer(image)
        image = cropper(image)
        if augment == True:
            image = flipper_1(flipper_2(image))
            image = rotater(image)        
        tensors_list.append(image)
        
    tensors = torch.stack(tensors_list)
    return tensors

In [ ]:
import torch
from torch import nn
from torch import optim
from torchvision import models

In [ ]:
class imageClassifier(nn.Module):
    """
        Given an image, return its class
    """
    
    def __init__(self, n_classes = 100):
        
        super().__init__()
        
        self.n_classes = n_classes
        pretrained = models.efficientnet_b5(pretrained=True)
        backbone = nn.Sequential(*list(pretrained.children())[0:2])
        for param in backbone.parameters():
            param.requires_grad = True
        
        self.scaler = transforms.Lambda(lambda x: x/255.)
        self.network = nn.Sequential(*[
            backbone,
            nn.Flatten(),
            nn.Linear(2048, n_classes),
            nn.Softmax(dim=0)
        ])
        
        self.optimizer = optim.Adam(self.parameters(), lr=1e-2)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        """
            Return the softmax prediction given an image
        """
        
        x = self.scaler(x)
        y_hat = self.network(x)
        
        return y_hat
    
    def fit(self, x, y):
        """
            Fit dataset
        """
        
        self.train()
        self.optimizer.zero_grad()
        
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)
        
        loss.backward()
        self.optimizer.step()
        
        return loss.detach().item()
    
    def predict(self, x):
        """
            Return the prediction for a given image
        """
        
        self.eval()
        with torch.no_grad():
            y_hat = self.forward(x)
            
        return(y_hat)

In [ ]:
classifierDataloader = DataLoader(range(len(X_classifier)), shuffle=True, batch_size=8)

In [ ]:
classifier = imageClassifier()

In [ ]:
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

classifier.to(device)

losses = []
for i in range(100):
    for idx in classifierDataloader:
        images = imageCollater(classifierDataset[idx]).to(device)
        labels = one_hot(torch.tensor(y_classifier_id[idx]), num_classes=100).to(device).float()

        loss = classifier.fit(images, labels)
        losses.append(loss)

    print(np.array(losses).mean())

In [ ]:
from torchvision.models import efficientnet_b5

efficientnet_model = efficientnet_b5(pretrained=True)

class MyEfficientNet(torch.nn.Module):
    def __init__(self):
        super(MyEfficientNet, self).__init__()
        self.first = torch.nn.Sequential(*list(efficientnet_model.children())[:-1])
        self.dropout = torch.nn.Dropout(p=0.4, inplace=True)
        self.classifier = torch.nn.Linear(2048, 100)

    def forward(self, x):
        x = self.first(x)
        x = x[:,:,0,0]
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [ ]:
import random
import sys

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn import preprocessing

import torch

from tqdm import tqdm

from torchvision import transforms
from torchvision.models import detection



from glob import glob
import os

In [ ]:
import importlib

importlib.reload(pp)
importlib.reload(utils)

In [ ]:
footprint_path = (
    "~/activities_data/"
    "hi__paris_2022_hackathon/"
    "final_challenge/"
    "datasets/"
    "car_models_footprint.csv"
)

car_path = (
    "~/activities_data/"
    "hi__paris_2022_hackathon/"
    "final_challenge/"
    "datasets/"
    "datasets_train/"
    "train_annotation/"
    "_annotation.csv"
)

imgs_path = (
    "activities_data/"
    "hi__paris_2022_hackathon/"
    "final_challenge/"
    "datasets/"
    "datasets_train/"
    "train/"
)

In [ ]:
train_proportion = 0.7

num_train = int(len(img_dataset) * train_proportion)
num_test = len(img_dataset) - num_train

train_set, val_set = torch.utils.data.random_split(img_dataset, [num_train, num_test])

In [ ]:
BATCH_SIZE = 1

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = {
    "frcnn-resnet": detection.fasterrcnn_resnet50_fpn,
    "frcnn-mobilenet": detection.fasterrcnn_mobilenet_v3_large_320_fpn,
    "retinanet": detection.retinanet_resnet50_fpn,
}

In [ ]:
box_set = pp.BoxDataset(img_dataset, retinanet, device, tol=4e-1, class_idx=3)

In [ ]:
from torchvision.models import efficientnet_b5

efficientnet_model = efficientnet_b5(pretrained=True)

In [ ]:
myefficientnet = MyEfficientNet().to(device)
myefficientnet.train()

In [ ]:
box_loader = torch.utils.data.DataLoader(box_set, batch_size=16, shuffle=True)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(efficientnet_model.parameters(), lr=1e-3)

img_size = 256 # 456 efficient net original training size
data_transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
])

n_epochs = 5

In [ ]:
import importlib

importlib.reload(pp)
importlib.reload(utils)

In [ ]:
losses = []
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:

    epoch_loss = 0

    for i_batch, ((imgs, boxes), labels) in enumerate(box_loader):

        mask = (boxes.sum(dim=1) > 0) & (labels != -1) # on prend les images qui n'ont pas ete predites (0,0,0,0) et dont le label est celui d'une voiture
        imgs = imgs[mask].to(device)
        boxes = boxes[mask].to(device)
        labels = labels[mask].to(device)

        if mask.sum() > 0:
            # attention : definit la taille des images qui rentrent dans le cnn
            imgs = utils.crop_imgs(imgs, boxes, data_transform)

            optimizer.zero_grad()

            preds = myefficientnet(imgs)

            loss = criterion(preds, labels)

            loss.backward()
            optimizer.step()

            pbar.set_postfix({
                'progress': '{0:.2f}'.format(100*i_batch/len(box_loader)) + '%',
                'batch loss': '{0:.2f}'.format(loss.item())
            })

            epoch_loss += loss.item()

    epoch_loss /= len(box_loader)
    pbar.set_description('{:.5e}'.format(epoch_loss))

    losses.append(epoch_loss)

In [ ]:
torch.save(myefficientnet, 'my_work/team-03-084-submission-matthieu/Code/trained_model.pth')

In [ ]:
plt.plot(losses)